In [1]:
from numba import njit, int16
from random import randint as rndint
from random import random as rnd
import numpy as np
from math import sqrt

In [2]:
@njit
# checks if position is in board
def isInBoard(position, dim):
    x,y = position
    return (0<=x<dim and 0<=y<dim)

@njit
# finds number of queens hits
def totHits(positions):
    hits = 0
    dim = len(positions)
    for i in range(dim):

        # moves up right
        (x,y) = (i+1, positions[i]+1)
        while isInBoard((x,y), dim):
            if positions[x] == y:
                hits+=1
            x+=1
            y+=1
        # moves down right
        (x,y) = (i+1, positions[i]-1)
        while isInBoard((x,y), dim):
            if positions[x] == y:
                hits+=1
            x+=1
            y-=1
        # moves up left
        (x,y) = (i-1, positions[i]+1)
        while isInBoard((x,y), dim):
            if positions[x] == y:
                hits+=1
            x-=1
            y+=1
        (x,y) = (i-1, positions[i]-1)
        # moves down left
        while isInBoard((x,y), dim):
            if positions[x] == y:
                hits+=1
            x-=1
            y-=1
    return hits

@njit
def evaluate(positions):
    return -totHits(positions)/2

In [3]:
@njit
def crossover(elem1, elem2, crossRatio):
    size = len(elem1)
    if rnd() >= crossRatio:
        return elem1.copy(), elem2.copy()
    crossPoint1 = rndint(0,size-1)
    # crossPoint2 = rndint(0,size-1)
    crossPoint2 = size-1
    while crossPoint1 == crossPoint2:
        crossPoint2 = rndint(0,size-1)
    # if crossPoint1 > crossPoint2:
    #     crossPoint1, crossPoint2 = crossPoint2, crossPoint1
    child1 = np.full(size, -1, int16)
    child2 = np.full(size, -1, int16)
    for i in range(crossPoint1, crossPoint2+1):
        child1[i] = elem1[i]
        child2[i] = elem2[i]
    free1 = crossPoint2+1
    free2 = crossPoint2+1
    for i in range(crossPoint2+1, crossPoint2+size+1):
        j = i % size
        if elem1[j] not in child2:
            free2 = free2 % size
            child2[free2] = elem1[j]
            free2 += 1
        if elem2[j] not in child1:
            free1 = free1 % size
            child1[free1] = elem2[j]
            free1 += 1
    return child1, child2

In [4]:
@njit
def tweak(positions):
    l = len(positions)
    x = rndint(0,l-1)
    y = rndint(0,l-1)
    while(x==y):
        y = rndint(0,l-1)
    positions[x], positions[y] = positions[y], positions[x]

@njit
#random starting population
def populationInitialize(populationSize, size):
    population = np.empty((populationSize, size), int16)
    for i in range(populationSize):
        element = np.arange(size, dtype=int16)
        for _ in range(10 * size):
            tweak(element)
        population[i] = element
    return population

@njit
def mutation(elem, mutationRatio):
    if rnd() < mutationRatio:
        tweak(elem)

In [5]:
@njit
def populationScores(flattenedPopulation, populationSize, size):
    population = flattenedPopulation.reshape((populationSize, size))
    scores = np.empty(populationSize, int16)
    for i in range(populationSize):
        scores[i] = evaluate(population[i])
    return scores

In [6]:
# rank based selection
@njit
def index(v):
    dis = 1 + (8*v)
    return int((-1+(sqrt(dis)))/2)

@njit
def rankSelection(flattenPopulation, populationSize, size, scores, totRanks):
    population = flattenPopulation.reshape((populationSize, size))
    pop_scores = sorted(zip(population, scores), key=lambda x: x[1])
    selected = np.empty((len(population), size), dtype=np.short)
    for i in range(len(population)):
        elemi = index(rndint(0, totRanks-1))
        selected[i] = pop_scores[elemi][0]
    return selected

In [7]:
@njit
def geneticAlgorithm(size, populationSize, iterations, crossRatio, mutationRatio):
    population = populationInitialize(populationSize, size)
    scores = populationScores(population.flatten(), populationSize, size)
    bestScore = scores.max()
    indexBestElement = np.where(scores == bestScore)[0][0]
    bestElement = population[indexBestElement]
    totRanks = int(populationSize * (populationSize + 1) / 2)
    for _ in range(iterations):
        selected = rankSelection(population.flatten(), populationSize, size, scores, totRanks)
        childs = np.empty((populationSize, size), dtype = np.short)
        for i in range(0, populationSize, 2):
            child1, child2 = crossover(selected[i], selected[i+1], crossRatio)
            mutation(child1, mutationRatio)
            mutation(child2, mutationRatio)
            childs[i] = child1
            childs[i+1] = child2
        population = childs
        scores = populationScores(population.flatten(), populationSize, size)
        bestChildScore = scores.max()
        if bestChildScore > bestScore:
            indexBestElement = np.where(scores == bestChildScore)[0][0]
            bestElement = population[indexBestElement]
            bestScore = bestChildScore
        if bestScore == 0:
            break
    return bestElement, bestScore

In [9]:
from time import perf_counter
size = 20
iterations = 10000
populationSize = 14
crossRatio =0.90
mutationRatio = 0.1
fails = 0
its = 30

start = perf_counter()
for _ in range(its):
    best, score = geneticAlgorithm(size, populationSize, iterations, crossRatio, mutationRatio)
    if score != 0:
        fails += 1
    print("value:", best, "score:", score)
end = perf_counter()

print("average time:", (end-start)/its)
print("fail ratio:", int(100*fails/its), "%")

value: [ 2 13 11 14  8  0 19  4  1 10 16 18 15  9  6  3  5  7 12 17] score: 0
value: [ 5 14 16 10  2  7 15 13 11 17  0  6  4  1 18  9 12  8 19  3] score: 0
value: [ 4 10 15  3  6 13 16  1  5 12  9 18  0  2 19  8 11  7 14 17] score: 0
value: [10 15  9 11 19 16  3  0  7 18  8  2 17 13  6  4  1  5 12 14] score: 0
value: [ 4  6  9 16 13  3 12 19  2  5 18 14  0 10  1  7 15 17 11  8] score: 0
value: [ 7  4  8 15 18 14  3 17 13  5  2  6 11  0 12  1 16 10 19  9] score: 0
value: [10  3 18  4 17 14  5  1  6  0  2 15 12 16 19  8 11 13  7  9] score: 0
value: [19  3 15 13 18 10  6  4  2  5 16 12  1 17  7 14 11  8  0  9] score: 0
value: [13 15  4  1 14 17  5  0  9 12  2 18  7 10 19 11 16  3  6  8] score: 0
value: [ 4 18 12  6  9 16  0  3 15 17  7 13 10  5  2 14 11  8 19  1] score: 0
value: [11  8 18  0 10  7  2 15 13 19  3  6  4 12  1 16 14 17  9  5] score: 0
value: [ 4 11 15 10 16  6  1 12  7  9  0  3 14 19  2 18 13  8  5 17] score: 0
value: [ 8 16 12 19  6 18  7 13  4  2 17 15  9  3  1 10 14 11  0